# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,portland,45.5234,-122.6762,30.51,41,0,0.89,US,1720502552
1,1,ushuaia,-54.8000,-68.3000,3.81,65,0,9.77,AR,1720502853
2,2,heyuan,23.7333,114.6833,33.78,53,18,2.23,CN,1720502854
3,3,polyarnyy,69.1989,33.4478,13.05,83,62,1.89,RU,1720502855
4,4,fisterra,42.9049,-9.2629,18.19,98,99,9.05,ES,1720502857


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
    
)

# Display the map plot
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

perfect_temp_df = city_data_df.loc[((city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)), :]
easy_breeze_df = perfect_temp_df.loc[(perfect_temp_df["Wind Speed"] < 4.5), :]
clear_skies_df = easy_breeze_df.loc[(easy_breeze_df["Cloudiness"] == 0), :]
# Drop any rows with null values
ideal_conditions_df = clear_skies_df.dropna()

# Display sample data
ideal_conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,bethel,41.3712,-73.4140,24.39,87,0,1.54,US,1720502912
61,61,fruita,39.1589,-108.7290,25.32,24,0,0.64,US,1720502938
66,66,hamilton,39.1834,-84.5333,24.85,81,0,2.06,US,1720502945
86,86,peace river,56.2501,-117.2860,21.17,88,0,1.54,CA,1720502978
120,120,morshansk,53.4435,41.8107,24.88,47,0,2.23,RU,1720503028
261,261,datca,36.7378,27.6842,26.50,69,0,2.28,TR,1720503229
293,293,selyatino,55.5151,36.9775,24.36,53,0,1.00,RU,1720503274
316,316,greenville,35.6127,-77.3663,26.02,93,0,3.60,US,1720503110
332,332,lewistown,40.5992,-77.5714,24.41,100,0,0.00,US,1720503331
344,344,saint-pierre,-21.3393,55.4781,23.09,49,0,1.54,RE,1720503348


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
43,bethel,US,41.3712,-73.4140,87,
61,fruita,US,39.1589,-108.7290,24,
66,hamilton,US,39.1834,-84.5333,81,
86,peace river,CA,56.2501,-117.2860,88,
120,morshansk,RU,53.4435,41.8107,47,
261,datca,TR,36.7378,27.6842,69,
293,selyatino,RU,55.5151,36.9775,53,
316,greenville,US,35.6127,-77.3663,93,
332,lewistown,US,40.5992,-77.5714,100,
344,saint-pierre,RE,-21.3393,55.4781,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000

limit = 1
params = {"categories":categories,
          "limit" : limit,
          "apiKey" : geoapify_key
}
# categories, filter, bias, limit, apikey
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city_latitude = row["Lat"]
    city_longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_longitude},{city_latitude},{radius}"
    params["bias"] = f"proximity:{city_longitude},{city_latitude}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    # name_address = f"{base_url}?categories={categories}&filter={filter}&bias={bias}&limit={limit}&apiKey={geoapify_key}"
    
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format

    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
fruita - nearest hotel: Balanced Rock Inn
hamilton - nearest hotel: North Vista Manor
peace river - nearest hotel: New Ridge Inn
morshansk - nearest hotel: No hotel found
datca - nearest hotel: Marphe Hotel
selyatino - nearest hotel: Русь
greenville - nearest hotel: Quality Inn
lewistown - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
kyustendil - nearest hotel: Стримон Гардън
pristen' - nearest hotel: No hotel found
murray - nearest hotel: Home2 Suites by Hilton Salt Lake City-Murray, UT
nema - nearest hotel: No hotel found
great falls - nearest hotel: La Quinta Inn & Suites
mount airy - nearest hotel: Knights Inn Mount Airy
berdigestyakh - nearest hotel: No hotel found
mao - nearest hotel: Hevresac
rindge - nearest hotel: Woodbound Inn
waddan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
43,bethel,US,41.3712,-73.4140,87,Hampton Inn Danbury
61,fruita,US,39.1589,-108.7290,24,Balanced Rock Inn
66,hamilton,US,39.1834,-84.5333,81,North Vista Manor
86,peace river,CA,56.2501,-117.2860,88,New Ridge Inn
120,morshansk,RU,53.4435,41.8107,47,No hotel found
261,datca,TR,36.7378,27.6842,69,Marphe Hotel
293,selyatino,RU,55.5151,36.9775,53,Русь
316,greenville,US,35.6127,-77.3663,93,Quality Inn
332,lewistown,US,40.5992,-77.5714,100,No hotel found
344,saint-pierre,RE,-21.3393,55.4781,49,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)